*Based on the report by George Frentzel*

## Max Flow Implementation
### PuLP

In [1]:
from pulp import *

In [2]:
VERTICES = ['S',2,3,4,5,6,'T']
EDGES = [('S',2), ('S',3), (2,3), (2,4), (2,5), (3,5),
(3,6), (4,'T'), (5,4), (5,6), (5,'T'),(6,'T')]

In [3]:
#Use a dictionary to associate capacities of each edge
capacity = {('S',2):1, ('S',3):7, (2,3):5, (2,4):6, (2,5):2,
            (3,5):2, (3,6):4, (4,'T'):5, (5,4):1, (5,6):4, (5,'T'):4, 
            (6,'T'):3}

In [4]:
#Create variables for each edge
vars = LpVariable.dicts("Edges",EDGES,None,None,LpInteger)

In [5]:
#Sets capacity of each edge to be at least 0, at most its capacity
for a in EDGES: vars[a].bounds(0,capacity[a])

In [6]:
#Creates an LpProblem object, given as maximization and a title
prob = LpProblem("Max␣flow␣Problem", LpMaximize)

In [7]:
#Defines objective function
prob += lpSum([vars[(i,j)] for (i,j) in EDGES if j=='T'])

In [8]:
# Creates all problem constraints
for n in VERTICES:
    if (n!='S' and n!='T'):
        prob += (lpSum([vars[(i,j)] for (i,j) in EDGES if j == n]) ==
        lpSum([vars[(i,j)] for (i,j) in EDGES if i == n]))

In [9]:
prob.writeLP("maxflowlp.lp")
prob.solve()
print('Max Flow =', value(prob.objective)) 
for (i,j) in EDGES: print((i,j), value(vars[(i,j)]))

Max Flow = 6.0
('S', 2) 1.0
('S', 3) 5.0
(2, 3) 0.0
(2, 4) 0.0
(2, 5) 1.0
(3, 5) 2.0
(3, 6) 3.0
(4, 'T') 0.0
(5, 4) 0.0
(5, 6) 0.0
(5, 'T') 3.0
(6, 'T') 3.0


### GurobiPy

In [10]:
from gurobipy import *

In [11]:
VERTICES = ['S',2,3,4,5,6,'T']
EDGES = tuplelist([('S',2), ('S',3), (2,3), (2,4), (2,5), (3,5),
(3,6), (4,'T'), (5,4), (5,6), (5,'T'),(6,'T')])
source = 'S'
sink = 'T'

In [12]:
#Use a dictionary to associate capacities of each edge
capacity = {('S',2):1, ('S',3):7, (2,3):5, (2,4):6, (2,5):2,
            (3,5):2, (3,6):4, (4,'T'):5, (5,4):1, (5,6):4, (5,'T'):4, 
            (6,'T'):3}

In [13]:
# Create optimization model
m = Model('maxflow')

Restricted license - for non-production use only - expires 2022-01-13


In [14]:
# create model variables
flowvar = {}
for i,j in EDGES:
    flowvar[i,j] = m.addVar()

In [15]:
#Need to update model after defining variables
m.update()
# define objective function
obfun = quicksum(flowvar[i,j] for i,j in EDGES.select(source,'*')) 
m.setObjective(obfun, GRB.MAXIMIZE)

In [16]:
#add capacity constraints and flow conservations constraints
for i,j in EDGES: 
    m.addConstr(flowvar[i,j]<=capacity[i,j])

In [17]:
for j in VERTICES:
    if j!=source and j!=sink:
        m.addConstr( quicksum(flowvar[i,j] for i,j in EDGES.select('*',j))
        == quicksum(flowvar[j,k] for j,k in EDGES.select(j,'*')))

In [20]:
# Compute optimal solution
m.optimize()
# Print solution
if m.status == GRB.Status.OPTIMAL: 
    solution = m.getAttr('x', flowvar) 
    print('\nOptimal␣flows:')
    for i,j in EDGES:
         print('%s -> %s: %g' % (i, j, solution[i,j]))

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 17 rows, 12 columns and 31 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+00]

Solved in 0 iterations and 0.01 seconds
Optimal objective  6.000000000e+00

Optimal␣flows:
S -> 2: 1
S -> 3: 5
2 -> 3: 0
2 -> 4: 1
2 -> 5: 0
3 -> 5: 2
3 -> 6: 3
4 -> T: 1
5 -> 4: 0
5 -> 6: 0
5 -> T: 2
6 -> T: 3


### PySCIPOpt

In [19]:
# imports
from pyscipopt import *

ModuleNotFoundError: No module named 'pyscipopt'

In [ ]:
VERTICES = ['S',2,3,4,5,6,'T']
EDGES = [('S',2), ('S',3), (2,3), (2,4), (2,5), (3,5),
        (3,6), (4,'T'), (5,4), (5,6), (5,'T'),(6,'T')]
source = 'S'
sink = 'T'
capacity = {('S',2):1, ('S',3):7, (2,3):5, (2,4):6, (2,5):2, (3,5):2, 
            (3,6):4, (4,'T'):5, (5,4):1, (5,6):4, (5,'T'):4, (6,'T'):3}

In [ ]:
m = Model("maxFlow") # create model

In [ ]:
f = {}
for i,j in EDGES:
    f[i,j] = m.addVar()

In [ ]:
obfun = quicksum(f[i,j] for i,j in EDGES if j == 'T') # objective function
m.setObjective(obfun, "maximize") # set objective

In [ ]:
# subject to edge capacities
for i,j in EDGES: 
    m.addCons(f[i,j]<=capacity[i,j])

In [ ]:
# subject to flow conservation
for k in VERTICES:
    if k!=source and k!=sink:
        m.addCons( quicksum(f[i,j] for i,j in EDGES if j == k)
                == quicksum(f[i,j] for i,j in EDGES if i == k))

In [ ]:
m.optimize() # optimize
# print optimal solution (if found)
if m.getStatus() == "optimal":
    print("Optimal value:", m.getObjVal())
    print("Solution:") 
    for i,j in EDGES:
        print('%s -> %s: %g' % (i, j, m.getVal(f[i,j])))
else: 
    print("Problem could not be solved to optimality")

### Pyomo

In [ ]:
# imports
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [ ]:
# select gurobi as the solver
opt = SolverFactory('gurobi')

In [ ]:
# initialize the model 
model = AbstractModel()
model.NODES = Set() # set of nodes 
model.EDGES = Set(within=model.NODES * model.NODES) # set of edges
model.s = Param(within=model.NODES) # source node
model.t = Param(within=model.NODES) # sink node
model.u = Param(model.EDGES) # edge capacities 
model.f = Var(model.EDGES, within=NonNegativeReals)

In [ ]:
# objective: maximize the flow in to the sink 
def NetFlow(model):   
    return sum(model.f[i, j] for (i, j) in model.EDGES if j == model.t)
model.maxFlow = Objective(rule=NetFlow , sense=maximize)

In [ ]:
# subject to: flow across every edge is less than the capacity 
def CapacityConstraint(model, i,j):
    return (model.f[i,j] <= model.u[i,j])
model.loadOnArc = Constraint(model.EDGES, rule=CapacityConstraint)

In [ ]:
# subject to: flow in to a node (aside from the source/sink) is equal to the flow out
def FlowConservation(model, k):
    if (k == model.s or k == model.t): return Constraint.Skip
    inFlow = sum(model.f[i, j] for (i, j) in model.EDGES if j == k) 
    outFlow = sum(model.f[i, j] for (i, j) in model.EDGES if i == k) 
    return (inFlow == outFlow)
model.flow = Constraint((model.NODES), rule=FlowConservation)

In [ ]:
instance = model.create_instance('data/maxflow.dat') 
results = opt.solve(instance)
instance.display()

### Google OR-Tools (Glop for LP and COIN-OR for LP and MIP)

In [ ]:
#imports
from ortools.linear_solver import pywraplp as OR

In [ ]:
VERTICES = ['S',2,3,4,5,6,'T']
EDGES = [('S',2), ('S',3), (2,3), (2,4), (2,5), (3,5),
        (3,6), (4,'T'), (5,4), (5,6), (5,'T'),(6,'T')]
source = 'S'
sink = 'T'
capacity = {('S',2):1, ('S',3):7, (2,3):5, (2,4):6, (2,5):2, (3,5):2, 
            (3,6):4, (4,'T'):5, (5,4):1, (5,6):4, (5,'T'):4, (6,'T'):3}

m = OR.Solver('maxFlow', OR.Solver.GLOP_LINEAR_PROGRAMMING) # define model

# define decision variables
f = {}
for i,j in EDGES:
    # NumVar creates continuous variable with ub, lb, and name 
    f[i,j] = m.NumVar(0, m.infinity(), ('(%s, %s)' % (i,j)))
    
# define objective function
m.Maximize(sum(f[i,j] for i,j in EDGES if j == sink)) # set objective

# subject to: flow across every edge is less than the capacity 
for i,j in EDGES:
    m.Add(f[i,j] <= capacity[i,j])

# subject to: flow in to a node (aside from the source/sink) is equal to the flow out
for k in VERTICES:
    if k != sink and k != source:
        flowIn = sum(f[i,j] for i,j in EDGES if j == k)
        flowOut = sum(f[i,j] for i,j in EDGES if i == k)
        m.Add(flowIn == flowOut)

m.Solve()
print('Solution:')
print('Objective value =', m.Objective().Value())
for i,j in EDGES:
    print(f[i,j].name(),':', f[i,j].solution_value())

## Dynamic Programming

In [ ]:
# input data
teams =32 
seeds =16 
pennies =100

seedcostdict = {1:25, 2:21, 3:18, 4:15, 5:12, 6:10, 7:8, 8:6, 9:5, 
                10:4 ,11:3 , 12:2 , 13:1 , 14:1 , 15:1 , 16:1}
tupledict= {1:('Michigan St',4,1), 2:('Fresno St',1,9), 
            3:('Gonzaga',2,12), 4:('Indiana St',1,13),
5:('Temple',3,11), 6:('Florida',1,3), 7:('Penn St',2,7), 
8:('North Carolina',1,2), 9:('Illinois',3,1), 10:('Charlotte',1,9), 
11:('Syracuse',2,5), 12:('Kansas',2,4), 13:('Notre Dame',1,6), 
14:('Missisippi',2,3), 15:('Butler',1,10), 16:('Arizona',5,2),
17:('Duke',6,1), 18:('Missouri',2,9), 19:('Utah St',1,12), 
20:('UCLA',2,4), 21:('USC',3,6), 22:('Boston College',2,3), 
23:('Iowa',1,7), 24:('Kentucky',2,2), 25:('Stanford',3,1), 
26:('St Josephs',1,9), 27:('Cincinatti',2,5), 28:('Kent St',1,13), 
29:('Georgia St',1,11), 30:('Maryland',4,3), 
31: ('Georgetown', 2, 10),  32: ('Hampton', 1, 15)}

# helpful functions
def name(team): return tupledict[team][0] # name of team
def wins(team): return tupledict[team][1] # number of wins of team 
def seed(team): return tupledict[team][2] # seed of team
def seedcost(seed): return seedcostdict[seed] # cost of seed
def cost(team): return seedcost(seed(team)) # cost of team

In [ ]:
# dynamic programming algorithm
def pack2(team,pennies):
    Table= [[0 for x in range(pennies+1)] for x in range(team+1)] 
    for i in range(team + 1):
        for w in range(pennies + 1): 
            if i == 0 or w == 0:
                Table[i][w] = 0 
            elif cost(i) <= w:
                Table[i][w]= max(wins(i)+Table[i-1][w-cost(i)],
                Table[i-1][w])
            else:
                Table[i][w] = Table[i - 1][w] 
    return Table[team][pennies]

def rectab(team,pennies):
    Table= [[0 for x in range(pennies+1)] for x in range(team+1)] 
    for i in range(team + 1):
        for w in range(pennies + 1): 
            if i == 0 or w == 0:
                Table[i][w] = 0
            elif cost(i) <= w:
                Table[i][w]= max(wins(i)+Table[i-1][w-cost(i)],
                Table[i-1][w]) 
            else:
                Table[i][w] = Table[i - 1][w]
    return Table

result = pack2 (32 ,100)
table=rectab(32,100) 
print(result)
i=32
pen =100

while i>0 and pen-cost(i)>=0:
    if table[i][pen] == table[i-1][pen-cost(i)] + wins(i):
        print(name(i))
        i=i-1 
        pen=pen-cost(i)
    else: i=i-1